# Telecom X — Parte 2 · Predicción de Churn (Versión B)

**Diferencias**: feature engineering extra, validación repetida (5×2), calibración seleccionada por Brier, y doble criterio de umbral de negocio.

In [ ]:
# === Dependencias (solo Colab) ===
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False
if IN_COLAB:
    !pip -q install imbalanced-learn >/dev/null


In [ ]:
# === Imports y setup ===
import os, json, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_recall_curve, roc_curve,
                             confusion_matrix, classification_report, brier_score_loss)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import mutual_info_classif
warnings.filterwarnings('ignore')
RANDOM_STATE = 17
np.random.seed(RANDOM_STATE)
USE_BALANCING = False
BALANCING_METHOD = 'ros'


In [ ]:
# === Parámetros ===
CSV_PATH = '/content/df_limpo.csv'
VALUE_RETAIN = 120.0
COST_CONTACT = 4.0
TOP_N = 300
os.makedirs('/content/reports', exist_ok=True)
os.makedirs('/content/data/interim', exist_ok=True)


## 1) Carga y feature engineering (versión B)

In [ ]:
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f'No se encontró df_limpo.csv en {CSV_PATH}')
df = pd.read_csv(CSV_PATH)
print('Shape df:', df.shape)
display(df.head(3))

assert 'Churn' in df.columns, "El CSV tratado debe incluir la columna 'Churn'"
y_raw = df['Churn'].astype('string').str.strip().str.lower()
mask = y_raw.isin(['yes','no'])
dfm = df.loc[mask].copy()
dfm['Churn_bin'] = y_raw.loc[mask].map({'yes':1,'no':0}).astype(int)

# --- Feature engineering extra ---
def safe_float(x):
    try:
        return float(str(x).strip())
    except Exception:
        return np.nan

monthly = None; total = None; tenure = None
for c in dfm.columns:
    lc = c.lower()
    if 'monthly' in lc and 'charge' in lc: monthly = c
    if 'total' in lc and 'charge' in lc: total = c
    if 'tenure' in lc: tenure = c

if total is not None:
    dfm['_TotalCharges_num'] = dfm[total].apply(safe_float)
if monthly is not None:
    dfm['_MonthlyCharges_num'] = pd.to_numeric(dfm[monthly], errors='coerce')
if tenure is not None:
    dfm['_tenure_num'] = pd.to_numeric(dfm[tenure], errors='coerce')

if 'internet.InternetService' in dfm.columns:
    dfm['has_fiber'] = dfm['internet.InternetService'].astype('string').str.contains('fiber', case=False, na=False).astype(int)
else:
    dfm['has_fiber'] = 0

if '_tenure_num' in dfm.columns:
    dfm['tenure_bin'] = pd.cut(dfm['_tenure_num'], bins=[-1,1,3,6,12,24,48,72,1e9],
                               labels=['0-1','2-3','4-6','7-12','13-24','25-48','49-72','72+'])
else:
    dfm['tenure_bin'] = 'unk'

if all(c in dfm.columns for c in ['_TotalCharges_num','_MonthlyCharges_num','_tenure_num']):
    dfm['charges_ratio'] = dfm['_TotalCharges_num'] / (dfm['_MonthlyCharges_num'] * (dfm['_tenure_num']+1))
else:
    dfm['charges_ratio'] = np.nan

X = dfm.drop(columns=['Churn','Churn_bin','customerID'], errors='ignore')
y = dfm['Churn_bin'].to_numpy()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
print('num_cols:', len(num_cols), '| cat_cols:', len(cat_cols))


## 2) Preprocesamiento y split

In [ ]:
try:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', ohe)
])

transformers = []
if len(num_cols)>0: transformers.append(('num', num_pipe, num_cols))
if len(cat_cols)>0: transformers.append(('cat', cat_pipe, cat_cols))
pre = ColumnTransformer(transformers)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)
print('Train/Test:', X_train.shape, X_test.shape)


## 3) Selección de variables (MI)

In [ ]:
Xt = pre.fit_transform(X)
mi = mutual_info_classif(Xt, y, discrete_features=False, random_state=RANDOM_STATE)
def get_feature_names(pre, num_cols, cat_cols):
    out = []
    if 'num' in pre.named_transformers_ and pre.named_transformers_['num'] is not None:
        out.extend(num_cols)
    if 'cat' in pre.named_transformers_ and pre.named_transformers_['cat'] is not None:
        oh = pre.named_transformers_['cat'].named_steps['onehot']
        out.extend(oh.get_feature_names_out(cat_cols).tolist())
    return out
feat_names = get_feature_names(pre, num_cols, cat_cols)
pairs = sorted(zip(feat_names, mi), key=lambda x: x[1], reverse=True)[:25]
plt.figure(figsize=(8,6))
plt.barh([n for n,_ in pairs[::-1]], [v for _,v in pairs[::-1]])
plt.title('Top 25 — Información Mutua'); plt.tight_layout(); plt.show()


## 4) Modelos + GridSearch (PR-AUC) con validación repetida (5×2)

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=RANDOM_STATE)
models = {
  'logreg': (LogisticRegression(max_iter=2000, class_weight='balanced'),
             {'model__C':[0.1,1,5,10]}),
  'rf': (RandomForestClassifier(n_estimators=400, random_state=RANDOM_STATE, n_jobs=-1, class_weight='balanced'),
         {'model__max_depth':[None,10,16], 'model__min_samples_leaf':[1,3,6]}),
  'hgb': (HistGradientBoostingClassifier(random_state=RANDOM_STATE),
          {'model__learning_rate':[0.05,0.1], 'model__max_depth':[None,10,14]})
}

best_est, best_name, best_score = None, None, -np.inf
scores = {}
for name, (est, grid) in models.items():
    pipe = Pipeline([('pre', pre), ('model', est)])
    gs = GridSearchCV(pipe, grid, scoring='average_precision', cv=cv, n_jobs=-1)
    gs.fit(X_train, y_train)
    scores[name] = gs.best_score_
    if gs.best_score_ > best_score:
        best_est, best_name, best_score = gs.best_estimator_, name, gs.best_score_
print('PR-AUC (CV repetida):')
for k,v in scores.items():
    print(f'{k:8s} -> {v:.4f}')
print(f'Mejor: {best_name} ({best_score:.4f})')


## 5) Calibración por Brier (isotónica vs Platt)

In [ ]:
pre_fitted = best_est.named_steps['pre'].fit(X_train, y_train)
Xtr = pre_fitted.transform(X_train)
Xte = pre_fitted.transform(X_test)
base = best_est.named_steps['model']

def fit_calibrated(base, Xtr, ytr, Xte, yte):
    out = {}
    try:
        iso = CalibratedClassifierCV(base, method='isotonic', cv=3).fit(Xtr, ytr)
        p_iso = iso.predict_proba(Xte)[:,1]; out['isotonic'] = (p_iso, brier_score_loss(yte, p_iso))
    except Exception:
        pass
    try:
        pl = CalibratedClassifierCV(base, method='sigmoid', cv=3).fit(Xtr, ytr)
        p_pl = pl.predict_proba(Xte)[:,1]; out['platt'] = (p_pl, brier_score_loss(yte, p_pl))
    except Exception:
        pass
    if not out:
        if hasattr(base, 'predict_proba'):
            p = base.fit(Xtr, ytr).predict_proba(Xte)[:,1]
        else:
            s = base.fit(Xtr, ytr).decision_function(Xte)
            s = (s - s.min())/(s.max()-s.min()+1e-9)
            p = s
        return p, 'none', brier_score_loss(yte, p)
    # elegir por menor Brier
    m, (pp, bs) = min(out.items(), key=lambda kv: kv[1][1])
    return pp, m, bs

y_score, cal_used, brier = fit_calibrated(base, Xtr, y_train, Xte, y_test)
roc = roc_auc_score(y_test, y_score)
pr  = average_precision_score(y_test, y_score)
print(f'Calibración: {cal_used} | Brier: {brier:.4f} | ROC-AUC: {roc:.4f} | PR-AUC: {pr:.4f}')
prec, rec, _ = precision_recall_curve(y_test, y_score)
fpr, tpr, _ = roc_curve(y_test, y_score)
plt.figure(figsize=(6,4)); plt.plot(rec, prec); plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('PR curve'); plt.show()
plt.figure(figsize=(6,4)); plt.plot(fpr, tpr); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC curve'); plt.show()


## 6) Umbral de negocio: máxima ganancia y no-pérdida

In [ ]:
def business_profit(y_true, p, thr, value=VALUE_RETAIN, cost=COST_CONTACT):
    y_pred = (p>=thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tp*value - (tp+fp)*cost

thr_grid = np.linspace(0.005, 0.5, 200)
profits = np.array([business_profit(y_test, y_score, t) for t in thr_grid])
idx = int(np.argmax(profits)); thr_best = float(thr_grid[idx]); profit_best = float(profits[idx])
thr_safe = float(thr_grid[np.argmax(profits>=0)]) if np.any(profits>=0) else float(thr_best)
print(f'Threshold óptimo (ganancia): {thr_best:.3f} | Ganancia: {profit_best:.2f}')
print(f'Threshold no-pérdida: {thr_safe:.3f}')
plt.figure(figsize=(6,4)); plt.plot(thr_grid, profits); plt.axvline(thr_best, ls='--'); plt.axvline(thr_safe, ls=':');
plt.xlabel('Threshold'); plt.ylabel('Profit'); plt.title('Curva de ganancia'); plt.show()

for thr in [thr_best, thr_safe]:
    print(f'--- Umbral {thr:.3f} ---')
    y_pred = (y_score>=thr).astype(int)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))


## 7) Artefactos

In [ ]:
metrics = {
  'best_model': best_name,
  'cv_best_pr_auc': float(best_score),
  'test_roc_auc': float(roc),
  'test_pr_auc': float(pr),
  'brier_score': float(brier),
  'calibration': cal_used,
  'thr_best': float(thr_best),
  'thr_safe': float(thr_safe),
  'profit_best': float(profit_best),
  'value_retain': float(VALUE_RETAIN),
  'cost_contact': float(COST_CONTACT)
}
with open('/content/reports/metrics_p2_v2.json','w') as f:
    json.dump(metrics, f, indent=2)
print('Guardado metrics_p2_v2.json')

lines = []
lines.append('# Informe de Resultados — Telecom X · Parte 2 (Versión B)')
lines.append('')
lines.append('## 1. Datos y validación')
lines.append(f'- Filas: {df.shape[0]} | Columnas: {df.shape[1]}')
lines.append(f'- Split 80/20 estratificado | CV repetida 5×2 (PR-AUC)')
lines.append('')
lines.append('## 2. Modelo y calibración')
lines.append(f'- Mejor por CV (PR-AUC): **{best_name}** → {best_score:.4f}')
lines.append(f'- Test: ROC-AUC={roc:.4f} | PR-AUC={pr:.4f} | Brier={brier:.4f} | cal={cal_used}')
lines.append('')
lines.append('## 3. Umbral de negocio')
lines.append(f'- Máxima ganancia: thr={thr_best:.3f} → {profit_best:.2f} (VALUE_RETAIN={VALUE_RETAIN}, COST_CONTACT={COST_CONTACT})')
lines.append(f'- No-pérdida: thr={thr_safe:.3f}')
with open('/content/reports/README_REPORT_P2_v2.md','w', encoding='utf-8') as f:
    f.write('\n'.join(lines))
print('Guardado README_REPORT_P2_v2.md')

salida = dfm.loc[X_test.index].copy()
salida['churn_score'] = y_score
salida['flag_churn_risk'] = (y_score >= thr_best).astype(int)
topN = salida.sort_values('churn_score', ascending=False).head(TOP_N)
top_path = '/content/reports/clientes_en_riesgo_topN_p2_v2.csv'
topN.to_csv(top_path, index=False)
print('Exportado', top_path)
topN.head(5)
